# Cancelling transactions in lusid

## Setup LUSID

In [27]:
import os
import json
from datetime import datetime, timedelta
import pytz

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

import pandas as pd

pd.set_option('display.max_columns', None)

secrets_path = '/Users/msingh/Projects/lusidws/secrets.json'

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name='LusidJupyterNotebook'
)

In [28]:
with open(r'config/build_transactions_mapping.json') as mapping_file:
    build_transactions_json_mapping = json.load(mapping_file)
#    display(build_transactions_json_mapping)
    
with open(r'config/get_holdings_mapping.json') as mapping_file:
    get_holdings_json_mapping = json.load(mapping_file)

## 1) Load default transactions into a new scope

In [29]:
scope = create_scope_id()
portfolio_code = 'EQUITY_UK'
scope

'3a46-ede9-c627-1e'

In [30]:
transaction_file = r'data/cancels/equity_transactions.csv'
transaction_df = pd.read_csv(transaction_file)
display(transaction_df)
transaction_df['portfolio_code'] = portfolio_code

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0002,Buy,02/01/2020,04/01/2020,12000,5,60000,GBP,ftse_tracker,NaN
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0000566504,SEDOL2,equity,EQ_1235,BHP,trd_0003,Buy,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN
3,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0000566504,SEDOL2,equity,EQ_1235,BHP,trd_0004,Buy,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN
4,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,SEDOL3,equity,EQ_1236,Barclays,trd_0005,Buy,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN
5,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,SEDOL3,equity,EQ_1236,Barclays,trd_0006,Buy,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN
6,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,SEDOL4,equity,EQ_1237,BP,trd_0007,Buy,02/01/2020,04/01/2020,100000,5,500000,GBP,ftse_tracker,NaN
7,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,SEDOL4,equity,EQ_1237,BP,trd_0008,Buy,02/01/2020,04/01/2020,100000,5,500000,GBP,ftse_tracker,NaN
8,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,SEDOL5,equity,EQ_1238,HSBC,trd_0009,Buy,02/01/2020,04/01/2020,20000,6,120000,GBP,ftse_tracker,NaN
9,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,SEDOL5,equity,EQ_1238,HSBC,trd_0010,Buy,02/01/2020,04/01/2020,20000,6,120000,GBP,ftse_tracker,NaN


In [31]:
seed_data_response = seed_data(
    api_factory,
    ['portfolios', 'instruments', 'transactions'],
    scope,
    transaction_file,
    'csv'
)

## 2) Lets check our holdings

In [32]:
txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [33]:
response = txn_port_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name']
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df.head(20)

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,default,LUID_00003D60,{},Aviva,EQUITY_UK,3a46-ede9-c627-1e,P,132000.0,132000.0,660000.0,GBP,660000.0,GBP,GBP
1,default,LUID_00003D61,{},BHP,EQUITY_UK,3a46-ede9-c627-1e,P,120000.0,120000.0,2160000.0,GBP,2160000.0,GBP,GBP
2,default,LUID_00003D68,{},Barclays,EQUITY_UK,3a46-ede9-c627-1e,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP
3,default,LUID_00003D67,{},BP,EQUITY_UK,3a46-ede9-c627-1e,P,200000.0,200000.0,1000000.0,GBP,1000000.0,GBP,GBP
4,default,LUID_00003D62,{},HSBC,EQUITY_UK,3a46-ede9-c627-1e,P,40000.0,40000.0,240000.0,GBP,240000.0,GBP,GBP
5,default,CCY_GBP,{},GBP,EQUITY_UK,3a46-ede9-c627-1e,B,3260000.0,3260000.0,3260000.0,GBP,3260000.0,GBP,GBP
6,default,LUID_00003D63,{},Morrisons,EQUITY_UK,3a46-ede9-c627-1e,P,360000.0,360000.0,720000.0,GBP,720000.0,GBP,GBP
7,default,LUID_00003D5Y,{},Tesco,EQUITY_UK,3a46-ede9-c627-1e,P,12000.0,12000.0,100000.0,GBP,100000.0,GBP,GBP
8,default,LUID_00003D65,{},Rightmove,EQUITY_UK,3a46-ede9-c627-1e,P,160000.0,160000.0,960000.0,GBP,960000.0,GBP,GBP
9,default,LUID_00003D5Z,{},vodafone,EQUITY_UK,3a46-ede9-c627-1e,P,900000.0,900000.0,900000.0,GBP,900000.0,GBP,GBP


## 3) What transactions make up our Aviva and BHP holdings?

In [34]:
build_transactions_response = txn_port_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date='2020-01-01', end_date='2021-12-31'
    ),
    property_keys=['Instrument/default/Name']
)

build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)

build_transactions_df.head(20)

,TransactionId,TransactionType,TransactionTypeDesc,ClientId,instrument_scope,LusidInstrumentId,TransactionDate,SettlementDate,Units,TransactionAmount,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TransactionToPortfolioRate,TransactionCurrency,strategy(3a46-ede9-c627-1e-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),ResultantHolding,InstrumentName,TransactionStatus,EntryDateTime,RealisedGainLoss,InstrumentCurrency
0,trd_0001,Buy,Purchase,EQ_1234,default,LUID_00003D60,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,600000.0,5.0,Price,600000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,120000.0,Aviva,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
1,trd_0002,Buy,Purchase,EQ_1234,default,LUID_00003D60,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12000.0,60000.0,5.0,Price,60000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,132000.0,Aviva,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
2,trd_0003,Buy,Purchase,EQ_1235,default,LUID_00003D61,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,1080000.0,18.0,Price,1080000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,60000.0,BHP,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
3,trd_0004,Buy,Purchase,EQ_1235,default,LUID_00003D61,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,1080000.0,18.0,Price,1080000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,120000.0,BHP,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
4,trd_0005,Buy,Purchase,EQ_1236,default,LUID_00003D68,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,150000.0,Barclays,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
5,trd_0006,Buy,Purchase,EQ_1236,default,LUID_00003D68,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,300000.0,Barclays,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
6,trd_0007,Buy,Purchase,EQ_1237,default,LUID_00003D67,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,100000.0,500000.0,5.0,Price,500000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,100000.0,BP,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
7,trd_0008,Buy,Purchase,EQ_1237,default,LUID_00003D67,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,100000.0,500000.0,5.0,Price,500000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,200000.0,BP,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
8,trd_0009,Buy,Purchase,EQ_1238,default,LUID_00003D62,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,20000.0,120000.0,6.0,Price,120000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,20000.0,HSBC,Active,2021-12-24 13:47:15.614122+00:00,[],NaN
9,trd_0010,Buy,Purchase,EQ_1238,default,LUID_00003D62,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,20000.0,120000.0,6.0,Price,120000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3a46-ede9-c627-1e,40000.0,HSBC,Active,2021-12-24 13:47:15.614122+00:00,[],NaN


## 4) Cancel one of the Aviva transactions

In [35]:
cancel_response = txn_port_api.cancel_transactions(
    scope=scope, code=portfolio_code, transaction_ids=['trd_0001']
)

In [36]:
first_cancel_datetime = cancel_response.as_at
print(f'The first cancel datetime request is: {first_cancel_datetime}')

The first cancel datetime request is: 2021-12-24 13:47:17.189525+00:00


## 5) Cancel two of the BHP transactions

In [37]:
cancel_response = txn_port_api.cancel_transactions(
    scope=scope, code=portfolio_code, transaction_ids=['trd_0003', 'trd_0004']
)

second_cancel_datetime = cancel_response.as_at
print(f'The second cancel datetime request is: {first_cancel_datetime}')

The second cancel datetime request is: 2021-12-24 13:47:17.189525+00:00


## 6) Check holdings again

In [38]:
response = txn_port_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=['Instrument/default/Name']
)

hodlings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,default,LUID_00003D60,{},Aviva,EQUITY_UK,3a46-ede9-c627-1e,P,132000.0,132000.0,660000.0,GBP,660000.0,GBP,GBP
1,default,LUID_00003D61,{},BHP,EQUITY_UK,3a46-ede9-c627-1e,P,120000.0,120000.0,2160000.0,GBP,2160000.0,GBP,GBP
2,default,LUID_00003D68,{},Barclays,EQUITY_UK,3a46-ede9-c627-1e,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP
3,default,LUID_00003D67,{},BP,EQUITY_UK,3a46-ede9-c627-1e,P,200000.0,200000.0,1000000.0,GBP,1000000.0,GBP,GBP
4,default,LUID_00003D62,{},HSBC,EQUITY_UK,3a46-ede9-c627-1e,P,40000.0,40000.0,240000.0,GBP,240000.0,GBP,GBP
5,default,CCY_GBP,{},GBP,EQUITY_UK,3a46-ede9-c627-1e,B,3260000.0,3260000.0,3260000.0,GBP,3260000.0,GBP,GBP
6,default,LUID_00003D63,{},Morrisons,EQUITY_UK,3a46-ede9-c627-1e,P,360000.0,360000.0,720000.0,GBP,720000.0,GBP,GBP
7,default,LUID_00003D5Y,{},Tesco,EQUITY_UK,3a46-ede9-c627-1e,P,12000.0,12000.0,100000.0,GBP,100000.0,GBP,GBP
8,default,LUID_00003D65,{},Rightmove,EQUITY_UK,3a46-ede9-c627-1e,P,160000.0,160000.0,960000.0,GBP,960000.0,GBP,GBP
9,default,LUID_00003D5Z,{},vodafone,EQUITY_UK,3a46-ede9-c627-1e,P,900000.0,900000.0,900000.0,GBP,900000.0,GBP,GBP


In [39]:
as_at_time = first_cancel_datetime - timedelta(milliseconds=10)

print(f'The current datetime is {datetime.now(tz=pytz.UTC)}...')
print(f'Getting holdings as at {as_at_time}...')
print(f'This is just the first cancellation datetime of {first_cancel_datetime}')

The current datetime is 2021-12-24 13:47:18.258373+00:00...
Getting holdings as at 2021-12-24 13:47:17.179525+00:00...
This is just the first cancellation datetime of 2021-12-24 13:47:17.189525+00:00


In [41]:
response = txn_port_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=['Instrument/default/Name'], as_at=as_at_time
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=False, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,instrument_scope,LusidInstrumentId,SubHoldingKeys,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,properties.Instrument/default/Name.effective_from,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,default,LUID_00003D60,{},Instrument/default/Name,Aviva,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,132000.0,132000.0,660000.0,GBP,660000.0,GBP,GBP
1,default,LUID_00003D61,{},Instrument/default/Name,BHP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,120000.0,120000.0,2160000.0,GBP,2160000.0,GBP,GBP
2,default,LUID_00003D68,{},Instrument/default/Name,Barclays,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP
3,default,LUID_00003D67,{},Instrument/default/Name,BP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,200000.0,200000.0,1000000.0,GBP,1000000.0,GBP,GBP
4,default,LUID_00003D62,{},Instrument/default/Name,HSBC,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,40000.0,40000.0,240000.0,GBP,240000.0,GBP,GBP
5,default,CCY_GBP,{},Instrument/default/Name,GBP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,B,3260000.0,3260000.0,3260000.0,GBP,3260000.0,GBP,GBP
6,default,LUID_00003D63,{},Instrument/default/Name,Morrisons,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,360000.0,360000.0,720000.0,GBP,720000.0,GBP,GBP
7,default,LUID_00003D5Y,{},Instrument/default/Name,Tesco,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,12000.0,12000.0,100000.0,GBP,100000.0,GBP,GBP
8,default,LUID_00003D65,{},Instrument/default/Name,Rightmove,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,160000.0,160000.0,960000.0,GBP,960000.0,GBP,GBP
9,default,LUID_00003D5Z,{},Instrument/default/Name,vodafone,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY_UK,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,3a46-ede9-c627-1e,0001-01-01 00:00:00+00:00,P,900000.0,900000.0,900000.0,GBP,900000.0,GBP,GBP
